## Кейс: Конкурсные списки абитуриентов

В 2021 году конкурсные списки абитуриентов стали зашифрованными, и вместо ФИО ВУЗы стали использовать номера СНИЛС или ID абитуриента в системе. Попробуем обойти систему и узнать, кто подал согласия на зачисления на нашу желаемую программу.

1) Выгружаем два списка: ранжированный со СНИЛС и список по алфавиту с ФИО

2) Ранжируем второй список и присоединяем к первому

3) Получаем ранжированные списки с ФИО и согласиями на зачисление

4) Полученный список можно выгрузить в xls файл

Attention: убедительная просьба не 

In [1]:
import numpy as np
import pandas as pd
import pandas_datareader.data as web
from IPython.display import display
import requests

In [2]:
# Выгрузим два списка:
url_1 = 'https://lk.ranepa.ru/pk/list.php?FT=1&FL=0&FK=&FC=&FB=&FO=&F1=149cf41f-6a2b-11e8-80f0-000c299441de&F2=66fef4e2-c613-11e5-80e4-005056a00b6e&F3=6e9ce087-c78b-4932-88a5-2628115de4b3&F4=d604534c-990e-11eb-a2d4-149ecf4ca97d#list_contract'
url_2 = 'https://lk.ranepa.ru/pk/list.php?FT=2&FL=0&FK=&FC=&FB=&FO=2&F1=149cf41f-6a2b-11e8-80f0-000c299441de&F2=66fef4e2-c613-11e5-80e4-005056a00b6e&F3=6e9ce087-c78b-4932-88a5-2628115de4b3&F4=d604534c-990e-11eb-a2d4-149ecf4ca97d'
html_1 = requests.get(url_1).content
html_2 = requests.get(url_2).content
df_list = pd.read_html(html_1)
sogl_df = pd.read_html(html_2)

In [3]:
# Для ранжированния списка нам нужно знать сумму баллов, давайте ее посчитаем
df = df_list[0]

def summa(x):
    if x.values[3]!=0 and x.values[4]!=0:
        a = max(x.values[3],x.values[4])
        return x.values[0:3].sum()+a
    else:
        return x.values.sum()
df = df.replace('-',0)
df.iloc[:,3:-1] = df.iloc[:,3:-1].applymap(lambda x: int(x))

df['Sum'] = df.iloc[:,3:-1].T.apply(lambda x: summa(x))

# Теперь выделим в отедльные таблицы БВИ и особую квоту

n_kvota = 1 # количество мест по особой квоте
df_ok = df[df['Номер дела'].str.contains('ОК')].sort_values('Sum', ascending=False,ignore_index=True)

if len(df_ok)>n_kvota:
    df_ok = df_ok.iloc[:n_kvota,:]

df_bvi = df[df['Номер дела'].str.contains('БВИ')].sort_values('Sum', ascending=False,ignore_index=True)

df = df[~(df['Номер дела'].str.contains('ОК')|df['Номер дела'].str.contains('БВИ'))] # Оставим только общий конкурс

In [4]:
# Теперь можем ранжировать наш список

k = np.max(df.groupby('Sum').count()['ФИО'])
df['Сумма без ИД'] = df['Sum']-df['Сумма баллов по индивидуальным достижениям']

def sort1(x,n=k,column='Математика'):
    a = x.nlargest(n,column).sort_values(column, ignore_index=True)
    return a

def sortirovka(x,n=k,column='Сумма без ИД'):
    a = x.nlargest(n,column).sort_values('Сумма без ИД',ascending=False,ignore_index=True)
    for i in range(len(x)-1):
        if len(a['Сумма без ИД'].unique())<len(a):
            a = a.groupby('Сумма без ИД').apply(lambda x: sort1(x,n=k,column='Математика')).droplevel('Сумма без ИД')
    return a

df = df.groupby('Sum').apply(lambda x: sortirovka(x,n=k,column='Сумма без ИД')).iloc[::-1].droplevel([0]).iloc[:,:-1]


In [5]:
spisok = pd.concat([df_bvi, df_ok,df], axis = 0, ignore_index = True)
spisok = spisok.iloc[:,1:-2]
with pd.option_context('display.max_rows', None):
    display(spisok)

,ФИО,Номер дела,Сумма баллов по индивидуальным достижениям,Русский язык,Математика,Информатика иИКТ (по выбору),Обществознание (по выбору)
0,Филимонова Дарья Эдуардовна,001704/БВИ,10,0,0,0,100
1,Тхагапсов Астемир Артурович,002043/БВИ,10,0,0,0,92
2,Пименова Екатерина Игоревна,000721/БВИ,5,0,0,0,95
3,Кудринская Наталья Михайловна,001355/БВИ,10,0,0,0,86
4,Шириязданов Оскар Ринатович,001361/БВИ,7,0,0,0,88
5,Козаченко Александр Денисович,000424/БВИ,5,0,88,0,0
6,Нечепорчук Злата Евгеньевна,000859/БВИ,5,0,0,0,88
7,Вильданова Эльвина Расимовна,000109/БВИ,7,0,0,0,83
8,Конзафаров Илья Андреевич,001422/БВИ,10,0,0,0,79
9,Теслицкий Макар Ильич,001570/БВИ,0,0,0,0,78


In [13]:
# Отберем людей, которые подали согласия на зачисление
spisok_sogl = spisok.copy()

spisok_sogl = pd.merge(sogl_df[0], spisok_sogl.iloc[11:,:].reset_index().iloc[:,1:])
spisok_sogl
spisok_sogl = spisok_sogl.iloc[:,[-2,1,4,5,6,7,8,9,10,13]]
spisok_sogl
spisok_sogl = spisok_sogl.loc[spisok_sogl['Согласие на зачисление']=='Подано заявление о согласии на зачисление на основные места'].reset_index().iloc[:,1:]
spisok_sogl

,ФИО,СНИЛС (ID),Сумма конкурсных баллов,Сумма баллов по вступительным испытаниям,Сумма баллов по индивидуальным достижениям,Русский язык,Математика,Информатика иИКТ (по выбору),Обществознание (по выбору),Согласие на зачисление
0,Шарипов Адиль Мауленович,01071125,292,285,7,95,100,0,90,Подано заявление о согласии на зачисление на о...
1,Ходжаев Валерий Манучехрович,16161162636 (01107012),290,283,7,88,98,0,97,Подано заявление о согласии на зачисление на о...
2,Иваницкая Екатерина Викторовна,18890052606 (01134526),279,274,5,100,82,0,92,Подано заявление о согласии на зачисление на о...
3,Кесарева Кристина Владиславовна,20664268259 (01093411),265,265,0,70,95,0,100,Подано заявление о согласии на зачисление на о...
4,Симонов Егор Дмитриевич,20277691162 (01156867),260,255,5,92,80,0,83,Подано заявление о согласии на зачисление на о...
5,Мельникова Виктория Леонидовна,15721770568 (01089220),258,253,5,94,80,0,79,Подано заявление о согласии на зачисление на о...
6,Муромцев Александр Валерьевич,19041660556 (01146431),252,247,5,82,82,0,83,Подано заявление о согласии на зачисление на о...
7,Досова Полина Юрьевна,18592337510 (01082820),252,247,5,90,74,0,83,Подано заявление о согласии на зачисление на о...
8,Ажгирей Вера Вадимовна,17336461675 (01097994),249,249,0,94,72,0,83,Подано заявление о согласии на зачисление на о...
9,Логинова Владислава Евгеньевна,17336461675 (01097994),249,249,0,94,72,0,83,Подано заявление о согласии на зачисление на о...


In [7]:
### При желании можно выгрузить результат в csv или xls:

# spisok.to_csv(path_or_buf = 'C:/Users/lizat/Desktop/Список бюджет.csv', encoding="windows-1251", sep=";")

In [8]:
# spisok_sogl.to_excel(excel_writer = 'C:/Users/lizat/Desktop/Список бюджет с согласиями.xls', encoding="windows-1251")